---
format:
  html:
    embed-resources: true
    code-fold: true
---

## Data Gathering

### Links to source code and API used
Link to the source of retrieval code: [Source Code](https://github.com/anly501/dsan-5000-project-shawnhxu/tree/main/codes/01-data-gathering)

Link to Riot's APIs: [Riot API](https://developer.riotgames.com/apis)

## Introduction of Data Source
__7 Rank Divisions Data Set in Python__<br><br>
The primary API (Application Programming Interface) that I will be accessing to receive data for my Data Science Project is the Riot API provided by Riot Games. The competitive video game that I would like to primarily research about and obtain data for is the game League of Legends (LoL). Since this game has been around since 2009 and that the game features many many different possible recordable information, I found LoL to be the perfect subject for my Data Science data-driven project. What's more is that the Riot API includes many options to retrieve information from.

![Riot Image [Source](https://www.riotgames.com/en/press)](images/riot.png)

One of the biggest reasons I chose LoL as the primary video game to research about is, again, the amount of information I can include in my dataset but also the ease of access in this information from the Riot Games API. Unlike many other games, the Riot API for LoL allows any users or developers to have free access to many functions. A function that really caught my attention and influenced me to choose the Riot API and LoL is the function that allows users to find lists of player names and their respective ID's. This is significantly useful because my research topic relates to how to determine a player's respective rank based on their benchmarks of match data. By being able to have access to these I am thus able to retrieve the player's most recent games and filtering their ranked games to get their most serious effort in terms of match data. 

![Kim "Deft" Hyuk-kyu at LoL Worlds 2022 Championship, Image [Source](https://www.theloadout.com/tournaments/league-of-legends-world-championship/lol-worlds)](images/deft_words.png)

From domain knowledge, the ladder queue called "Ranked Solo/Duo 5v5" is the most commonly played queue and is considered the best way to compare a player's skill rating in most regions. This queue features the option to play a match of five players versus 5 players of equal skill rating. This data is in fact included in the API when retrieving a certain match's data. Each match has information about which participants are in the match and their respective 'puuid'. Then each participant would have their own information for each category of data which can be easily indexed after retrieving the match data.

## My Retrieval Process
So the process for retrieving player match that I used for this project is as follows:

```python
#Packages Used:
import requests
import pandas as pd
import numpy as np
from time import sleep
```

__<u>First</u>__, I used the function that the Riot API offers which gives me a list of player names and their account ID's in a certain rank. So in order to get a descriptive data set, I decided to retrieve an equal amount of players from each rank division at the same tier of that rank division. More specifically, I manually chose the ranks "Iron, Bronze, Silver, Gold, Platinum, Emerald, and Diamond" and the tier II for each rank division. So I requested a list of 205 player account names/their respective player ID's for each rank (Iron II, Bronze II, Silver II, etc.) through my ```get_league()``` function:

In [ ]:
#function get all summoner_names in a certain division
#like GOLD, DIAMOND, PLATINUM
#param: division as string, riot API key
#return: puuid as string
def get_league(division, riot_api_key):
    request_url = ("https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/" + division + 
                   "/II?page=1" + "&api_key=" + riot_api_key) #create correct request url
    resp = requests.get(request_url) #get response after requesting from url
    players = resp.json() #get dict of page 1 of players in certain division
    league_list = [] #init empty list to append player summoner names
    for dict in players: #loop thru each player in the dict
        player = dict['summonerName'] #add the summoner name to empty list
        league_list.append(player)
    return league_list

__<u>Secondly</u>__ after getting a list of players in their rank, I looped through the player usernames and requested the player's information on the Riot API in order to retrieve their ```'puuid'``` as this is required in order to retrieve data from their played matches. So as I loop through the player usernames I get their puuid. To do this I made a function called ```get_puuid()```:

In [ ]:
#function get puuid from summoner name
#param: player name as string, riot API key
#return: puuid as string
def get_puuid(summoner_name, riot_api_key): 
    request_url = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + summoner_name
    request_url_api = request_url + "?api_key=" + riot_api_key #create request url
    resp = requests.get(request_url_api) #get response
    if resp.status_code == 200:
        player_info = resp.json() #parse player info
        if 'puuid' in player_info: #only looking for dictionary where 'puuid' exists
            puuid = player_info['puuid'] #retrieve only puuid to use later
    return puuid

__<u>Thirdly</u>__, I take the ```'puuid'``` for each player and get their most recent 10 matches. So now, for each player puuid, I get a list of 10 match ID's that I will later loop through as well. The ```get_matches()``` function performs this:

In [ ]:
#function to get 10 match ID's with puuid
#param: puuid as string, riot API key
#return: match id's as list of strings 
def get_matches(puuid, riot_api_key): 
    match_url = (
        "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/" +
        puuid +
        "/ids?start=0&count=10" +
        "&api_key=" +
        riot_api_key
    ) #create request url
    resp = requests.get(match_url) #request with url
    matches = resp.json() #parse it
    return matches #we get the 10 most recent matches of a player

__<u>Finally</u>__, I loop through the 10 most recent matches ID's and take the ID's to put into another Riot API function that takes a match ID as a parameter and outputs/returns a very large dictionary containing the recordable match data performed by the players during the match as well as other metadata that I can use for organizationaly purposes. That being said, each match data has a something called a ```queueId```. This is important because I only want to retrieve information from the queue ladder called ```'Ranked Solo/Duo 5v5'```. Going into the Riot API documentation, I find that the ```queueId``` I want for this task would be a ```queueId``` of ```420```. So by looping through the match data, I filter out the matches that have a ```queueId``` of ```420``` and ignore all other matches. By doing so, the method I chose to replace those rows of the wrong queue ladder type would be to simply append rows of ```NA```'s. This would have to be removed later in Data Cleaning. Also, I added the respective rank division in a separate column of each data set. This might be helpful if I were to concatenate all the ranks into a single data frame. 

First is the function to get the most recent 10 match ID's. The function is called ```get_matches()```:

In [ ]:
#function to get 10 match ID's with puuid
#param: puuid as string, riot API key
#return: match id's as list of strings 
def get_matches(puuid, riot_api_key): 
    match_url = (
        "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/" +
        puuid +
        "/ids?start=0&count=10" +
        "&api_key=" +
        riot_api_key
    ) #create request url
    resp = requests.get(match_url) #request with url
    matches = resp.json() #parse it
    return matches #we get the 10 most recent matches of a player

Then is the function to get the specific match data that I want. This was done with the function ```player_data()```:

In [ ]:
#function to get specific types of data from match data
#param: puuid as string, match_data as dict
#return: win(true/false), kills, deaths, 
#damageDealtToObjectives,  damageDealtToTurrets, visionScorePerMinute,
#totalDamageDealt, totalDamageTaken, totalMinionsKilled, 
#goldEarned, individualPosition, time in minutes
##these variables were determined by domain knowledge
##columns = ['win','kills','deaths','dmgObj','dmgTurr','vision_score','totalDmg','totalDmgTaken','totalMinions','gold','lane','time']
def player_data(puuid, match_data):
    match = np.nan
    if 'info' in match_data: #only looking for the data where 'info' dictionary exists
        if match_data['info']['queueId'] == 420:
            player_index = match_data['metadata']['participants'].index(puuid) #get index of player in the match
            #because the dictionary is made according to the index of the player's puuid in the initial dict
            win = match_data['info']['participants'][player_index]['win'] #win or loss (true/false)
            kills = match_data['info']['participants'][player_index]['kills'] #number of kills in the match
            deaths = match_data['info']['participants'][player_index]['deaths'] #number of deaths in the match
            dmgObj = match_data['info']['participants'][player_index]['damageDealtToObjectives'] #dmg to objectives in the match
            dmgTurr = match_data['info']['participants'][player_index]['damageDealtToTurrets'] #dmg dealt to objectives in the match
            vision_score = match_data['info']['participants'][player_index]['visionScore'] #vision score in the match
            totalDmg = match_data['info']['participants'][player_index]['totalDamageDealt'] #total damage dealt in the match
            totalDmgTaken = match_data['info']['participants'][player_index]['totalDamageTaken'] #total damage taken in the match
            totalMinions = match_data['info']['participants'][player_index]['totalMinionsKilled'] #total minions killed in the match
            gold = match_data['info']['participants'][player_index]['goldEarned'] #amount of gold earned in the match
            position = match_data['info']['participants'][player_index]['individualPosition'] #which position/lane the player played in that match
            time = (match_data['info']['gameDuration'])/60 #match duration in minutes
            match = pd.Series([win, kills, deaths, dmgObj, dmgTurr, vision_score, totalDmg, totalDmgTaken, totalMinions, gold, position, time],
                          index=['win','kills','deaths','dmgObj','dmgTurr','vision_score','totalDmg','totalDmgTaken','totalMinions','gold','position','time'])
    return match

Finally, the function ```get_match_data()``` that uses the ```player_data()``` function to get data and store it into a dataframe for dataset building:

In [ ]:
#function to get match DATA with a list of match id
#param: puuid as string, match ID's as list of strings, riot API key, empty df
#empty df should have columns with these names in this order:
#['win','kda','dmgObj','gold','vision_score','lane','time','game_type']
#return: updated df with data
def get_match_data(puuid, matches, riot_api_key, empty_df):
    for match in matches:
        match_data_url = (
            "https://americas.api.riotgames.com/lol/match/v5/matches/" +
            match + 
            "?api_key=" +
            riot_api_key
        ) #create request url
        match_data = requests.get(match_data_url) #request the url
        match_data_parse = match_data.json() #parse the response
        data_series = player_data(puuid, match_data_parse) #get all desired match variables/data
        data_series2 = pd.DataFrame([data_series]) #change series to df
        empty_df = pd.concat([empty_df, data_series2], ignore_index=True) #concat to empty df to add to the dataset

    return empty_df

Now putting this together, the steps to build this dataset:

1. Use ```get_league()``` function to get 205 summoner names in a given division
2. Loop over the list returned by ```get_league()``` to run through each player
3. Get ```'puuid'``` of each player
4. Get recent ```10``` matches of each player
5. Get match data if the match is ```ranked solo/duo 5v5``` on ```summoner's rift```
6. Output as dataframe


Sample code that it put it all together for ```Iron II``` rank:

In [ ]:
#init empty df with desired variables for IRON
df_iron = pd.DataFrame(columns=['win','kills','deaths','dmgObj','dmgTurr','vision_score','totalDmg','totalDmgTaken','totalMinions','gold','position','time'])

#create list of all summoner names in division iron tier 2 (IRON II)
league_list = get_league('IRON', riot_api_key)
#run through the necessary functions to update df with desired match data
for summoner in league_list:
    puuid_summoner = get_puuid(summoner, riot_api_key) #getting puuid of player
    matches_summoner = get_matches(puuid_summoner, riot_api_key) #get recent 10 matches with puuid
    df_iron = get_match_data(puuid_summoner, matches_summoner, riot_api_key, df_iron) #update df with match data
#to categorize this data of Iron II players, I will manually add a column with their respective rank
df_iron['rank'] = 'iron'

After retrieving and storing it in data frames in my code, the only thing left is to write the data frames into csv excel files. After doing that, I am left with 7 different data sets from this process which is one excel file for each respective rank division at tier II. 

I did another process after this later for text data. I used the same exact process but this time only extracting individual player positions they are playing in the match and also their champion (fictional character within the game) that they chose. This text data will later be used for classification. This can be found in the [data folder](https://github.com/anly501/dsan-5000-project-shawnhxu/tree/main/data/01-raw-riot-data) as well labeled with 'text data'.
<br>

__Challenger Data Set in R__ <br><br>
Now that I have this process of acquiring data sets, I decided to retrieve more data through R from the same API but using different functions. The main difference this time is that I am looking for data from extremely high ranked players in League of Legends. 

So how this works is that after the rank division ```"Diamond"```, there are 3 more divisions that dynamically change based on the total sum of a player's skill rating points. The points are determined by Riot's Skill Rating algorithm. So after ```"Diamond"``` comes ```"Masters"``` then ```"Grandmaster"``` and finally ```"Challenger"```. Each of those ranks don't have tiers and are differentiated at dynamically changing skill rating points thresholds. For example, the threshold to become a ```"Challenger"``` in rank is to reach a total of ```800 League Points``` (hypothetically).

Knowing that, I decided to retrieve more data on players for League of Legends but this time I will be inspecting the match data of the top 300 players on the leaderboard and their recent match data. This is possible because Riot API has a separate function specifically for players on the ```"Challenger"``` ranked ladder. And so using the same process as before for other players, I created a dataset filled with match data from the top 300 players in <u>North America</u> and their recent ```'Ranked Solo/Duo 5v5'``` matches. Again, filtering the matches with ```queueId = 420``` and storing the desired match data into a data frame. This time, everyone is ranked ```"Challenger"``` so I will not be labeling each match with the rank.

So now, I have an additional data set in the form of a csv file for ```"Challenger"``` players.

This was all done in R in the .rmd file which can be found [here](https://github.com/anly501/dsan-5000-project-shawnhxu/blob/main/codes/01-data-gathering/riot_api_retrieval_R.Rmd).



__Text Data Set Retrieval (Python)__ <br>

In addition to the numerical record data that I retrieved, I also retrieved a simple text dataset from the Riot API as well. I followed the exact same process as I did with my Python code I showcased. The changes I made were made in the ```player_data()``` function where I extracted different categories of player data. Specifically, I extracted the ```'championName'``` and the ```'individualPosition'``` match data. ```'championName'``` is the fictional character being played by the player and ```'individualPosition'``` is the position that the player is playing in that specific match. 

Here is the altered code for ```player_data()```:

In [ ]:
def player_data(puuid, match_data):
    match = np.nan
    if 'info' in match_data: #only looking for the data where 'info' dictionary exists
        if match_data['info']['queueId'] == 420: #only ranked matches are wanted
            player_index = match_data['metadata']['participants'].index(puuid) #get index of player in the match
            #because the dictionary is made according to the index of the player's puuid in the initial dict
            champion = match_data['info']['participants'][player_index]['championName'] #the champion played
            position = match_data['info']['participants'][player_index]['individualPosition'] #number of kills in the match

            match = pd.Series([champion, position],
                          index=['champion','position'])
    return match